In [ ]:
import pandas as pd

df1 = pd.read_csv("Batch_2_train_labels_clean.csv")
df2 = pd.read_csv("Batch_1_train_labels_clean.csv")

df3 = pd.read_excel("image_labels_batch_3.xlsx")
df4 = pd.read_excel("image_labels_batch_4.xlsx")




In [2]:


df = pd.concat([df1, df2, df3], ignore_index=True)



In [3]:
print(df)

                                  image_name           category  \
0       image_874729307_product_74244625.jpg          PC Gaming   
1       image_874730792_product_49132716.jpg           Nintendo   
2       image_874731199_product_74093483.jpg        PlayStation   
3       image_874731370_product_49948298.jpg           Nintendo   
4      image_881487935_product_126111824.jpg           Nintendo   
..                                       ...                ...   
346  image_1100063096_product_1711731907.jpg          PC Gaming   
347  image_1100063685_product_1711736000.jpg          PC Gaming   
348  image_1100064358_product_1711734478.jpg          PC Gaming   
349  image_1119523636_product_2000192031.jpg  Réalité Virtuelle   
350  image_1139162966_product_2379124886.jpg           Nintendo   

               subcategory  
0                  Jeux PC  
1    Jeux Game Boy Advance  
2                 Jeux PS3  
3    Jeux Game Boy Advance  
4         Jeux Nintendo DS  
..                   

In [14]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
df['category_encoded'] = label_encoder.fit_transform(df['category'])


In [15]:
import tensorflow as tf
import pandas as pd
import os
import numpy as np

# Charger le DataFrame contenant les informations des images et des labels
# df = pd.read_csv("ton_fichier.csv")  # Assure-toi que df est déjà chargé dans ton environnement (avec les colonnes "image_name" et "category")

# Définir le dossier contenant les images
image_dir = "Projet_3/batchs_équilibrés/batch_entrainement/"

# Fonction pour charger une image et son label avec TensorFlow
def load_image(img_name, label):
    # Construire le chemin complet de l'image avec tf.strings.join
    img_path = tf.strings.join([image_dir, img_name])  # Joindre le répertoire et le nom de l'image
    img = tf.io.read_file(img_path)  # Lire l'image avec TensorFlow
    img = tf.image.decode_jpeg(img, channels=3)  # Décoder l'image (JPEG) en un Tensor
    img = tf.image.resize(img, [224, 224])  # Redimensionner l'image à 224x224
    img = img / 255.0  # Normalisation des pixels entre 0 et 1
    return img, label

# Convertir les noms d'images et labels en TensorFlow Dataset
image_names = df["image_name"].values
labels = df["category_encoded"].values

# Créer un dataset TensorFlow à partir des chemins d'images et des labels
dataset = tf.data.Dataset.from_tensor_slices((image_names, labels))

# Appliquer la fonction de chargement d'image et label avec map
dataset = dataset.map(lambda x, y: load_image(x, y))

# Diviser en batchs, mélanger et pré-charger pour améliorer les performances
dataset = dataset.batch(32).shuffle(1000).prefetch(tf.data.experimental.AUTOTUNE)

In [19]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

# Définir le modèle CNN (Convolutional Neural Network)
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(len(np.unique(labels)), activation='softmax')  # Nombre de classes
])

# Compiler le modèle
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Entraîner le modèle
history = model.fit(dataset, epochs=10)

# Sauvegarder le modèle
model.save("model.h5")

# Affichage des résultats de l'entraînement
print("Entraînement terminé. Le modèle a été sauvegardé sous 'model.h5'.")

# Si tu souhaites visualiser l'historique de l'entraînement, voici comment :
# Vérifier que les clés 'loss' et 'accuracy' existent dans l'historique
if 'accuracy' in history.history:
    acc = history.history['accuracy']
else:
    acc = history.history['accuracy_1']  # Selon la version de TensorFlow, ça peut être accuracy_1

# Tracer la courbe de perte
plt.plot(history.history['loss'], label='Perte (loss)')
plt.plot(acc, label='Précision (accuracy)')
plt.title("Courbes de l'entraînement")
plt.xlabel("Époques")
plt.ylabel("Valeur")
plt.legend()
plt.show()


Epoch 1/10


UnicodeDecodeError: 'utf-8' codec can't decode byte 0x92 in position 191: invalid start byte

In [ ]:
import pandas as pd
import tensorflow as tf
import numpy as np
import os

# Lire les fichiers CSV en utilisant un encodage correct
try:
    df1 = pd.read_csv("Batch_2_train_labels_clean.csv", encoding='utf-8')
    df2 = pd.read_csv("Batch_1_train_labels_clean.csv", encoding='utf-8')
except UnicodeDecodeError:
    df1 = pd.read_csv("Batch_2_train_labels_clean.csv", encoding='utf-8', errors='ignore')
    df2 = pd.read_csv("Batch_1_train_labels_clean.csv", encoding='utf-8', errors='ignore')

df3 = pd.read_excel("image_labels_batch_3.xlsx", engine='openpyxl')
df4 = pd.read_excel("image_labels_batch_4.xlsx", engine='openpyxl')

# Fusionner les DataFrames
df = pd.concat([df1, df2, df3, df4], ignore_index=True)

# Encoder les catégories en valeurs numériques
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
df['category_encoded'] = label_encoder.fit_transform(df['category'])

# Définir le dossier contenant les images
image_dir = "batchs_équilibrés\\batch_entrainement - Copie\\"


def load_image(img_name, label):
    try:
        # Utiliser tf.py_function pour s'assurer que l'on travaille avec une chaîne et non un tenseur
        img_name_str = tf.py_function(lambda x: x.numpy().decode('utf-8'), [img_name], Tout=tf.string)

        img_path = tf.strings.join([image_dir, img_name_str])  # Construire le chemin complet
        img = tf.io.read_file(img_path)  # Lire l'image
        img = tf.image.decode_jpeg(img, channels=3)  # Décoder l'image
        img = tf.image.resize(img, [224, 224])  # Redimensionner l'image
        img = img / 255.0  # Normaliser l'image
        return img, label

    except Exception as e:
        print(f"Erreur lors du chargement de l'image {img_name}: {e}")
        return tf.zeros([224, 224, 3]), label  # Retourner une image vide au lieu de None






def filter_invalid_images(img, label):
    """Filtrer les images invalides."""
    return img is not None

# Convertir les noms d'images et labels en TensorFlow Dataset
image_names = df["image_name"].values
labels = df["category_encoded"].values

# Créer un dataset TensorFlow à partir des chemins d'images et des labels
dataset = tf.data.Dataset.from_tensor_slices((image_names, labels))

# Appliquer la fonction de chargement d'image et label avec map
dataset = dataset.map(lambda x, y: load_image(x, y))

# Filtrer les images invalides
dataset = dataset.filter(filter_invalid_images)

# Diviser en batchs, mélanger et pré-charger pour améliorer les performances
dataset = dataset.batch(32).shuffle(1000).prefetch(tf.data.experimental.AUTOTUNE)

# Modèle simplifié pour tester
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(16, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(len(np.unique(labels)), activation='softmax')
])

# Compiler et tester l'entraînement avec un modèle plus simple
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

history = model.fit(dataset, epochs=1)


# Sauvegarder le modèle
model.save("model.h5")

# Affichage des résultats de l'entraînement
print("Entraînement terminé. Le modèle a été sauvegardé sous 'model.h5'.")


UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe9 in position 194: invalid continuation byte

In [19]:
# Vérifier que le répertoire existe et lister les fichiers
image_dir = "batchs_équilibrés\\batch_entrainement\\"

print(f"Vérification des fichiers dans {image_dir} :")
print(os.listdir(image_dir))  # Affiche les fichiers présents dans le répertoire


Vérification des fichiers dans batchs_équilibrés\batch_entrainement\ :
['image_1000493202_product_353771692.jpg', 'image_1004469307_product_299520596.jpg', 'image_1016066453_product_493545596.jpg', 'image_1025231310_product_436182917.jpg', 'image_1027953365_product_301940235.jpg', 'image_1027961004_product_578256383.jpg', 'image_1032783160_product_630642988.jpg', 'image_1039267141_product_646789701.jpg', 'image_1041335988_product_825941638.jpg', 'image_1041415174_product_826453416.jpg', 'image_1041748138_product_832656848.jpg', 'image_1044577992_product_861269059.jpg', 'image_1049879107_product_957011147.jpg', 'image_1051593_product_1283013.jpg', 'image_1053085531_product_1022623650.jpg', 'image_1053259382_product_1028931562.jpg', 'image_1053519738_product_1033730640.jpg', 'image_1058972548_product_1089685811.jpg', 'image_1062523736_product_301815907.jpg', 'image_1068150012_product_1132613553.jpg', 'image_1068227670_product_1190608934.jpg', 'image_1068342_product_3108655.jpg', 'image_1

In [ ]:
# Afficher les premières lignes pour vérifier les noms d'images
print(df['image_name'].head())


# Vérifier les noms d'images pour des caractères spéciaux ou invisibles
for name in df['image_name']:
    try:
        name = str(name).encode('utf-8', 'ignore').decode('utf-8', 'ignore')
    except UnicodeDecodeError:
        print(f"Erreur avec le nom de fichier : {name}")


0     image_874729307_product_74244625.jpg
1     image_874730792_product_49132716.jpg
2     image_874731199_product_74093483.jpg
3     image_874731370_product_49948298.jpg
4    image_881487935_product_126111824.jpg
Name: image_name, dtype: object


In [33]:
# Vérification de l'existence des fichiers images
for img_name in df["image_name"]:
    img_path = os.path.join(image_dir, img_name)
    if not os.path.isfile(img_path):
        print(img_path)
        print(f"Erreur: L'image suivante est manquante : {img_name}")


In [59]:
import os

image_dir = "batchs_équilibrés/batch_entrainement - Copie/"

missing_files = []
for name in df['image_name']:
    img_path = os.path.join(image_dir, name)
    if not os.path.exists(img_path):
        missing_files.append(name)

if missing_files:
    print("🚨 Images manquantes :", missing_files)  # Affiche les 10 premières pour éviter trop de spam
else:
    print("✅ Tous les fichiers existent !")


🚨 Images manquantes : ['image_891725531_product_138457897.jpg', 'image_23276830_product_1533426.jpg', 'image_40504830_product_1788665.jpg', 'image_486903891_product_51173981.jpg', 'image_651743714_product_54844666.jpg', 'image_830635508_product_63004531.jpg', 'image_833130849_product_61985267.jpg', 'image_844644693_product_51529973.jpg', 'image_849076215_product_80378088.jpg', 'image_858567869_product_92021901.jpg', 'image_859921037_product_2923354.jpg', 'image_874537596_product_107645092.jpg', 'image_874538150_product_102311725.jpg', 'image_891727975_product_138459513.jpg', 'image_892247180_product_139269024.jpg', 'image_901354345_product_146713592.jpg', 'image_921363344_product_168669058.jpg', 'image_933233976_product_190091431.jpg', 'image_933234188_product_190091335.jpg', 'image_933252862_product_190106248.jpg', 'image_1018013263_product_508622373.jpg', 'image_1027902744_product_577491837.jpg', 'image_1203606331_product_3280203098.jpg', 'image_1216045204_product_2843201888.jpg', 'i

In [37]:
# Vérifier si le dataset lui-même est correct
image_names = df["image_name"].values
labels = df["category_encoded"].values

dataset = tf.data.Dataset.from_tensor_slices((image_names, labels))

# Afficher quelques éléments du dataset avant de les passer dans la fonction de chargement
for img_name, label in dataset.take(5):  # Prendre les 5 premiers éléments
    print(f"Nom de l'image: {img_name}, Label: {label}")


Nom de l'image: b'image_874729307_product_74244625.jpg', Label: 5
Nom de l'image: b'image_874730792_product_49132716.jpg', Label: 3
Nom de l'image: b'image_874731199_product_74093483.jpg', Label: 6
Nom de l'image: b'image_874731370_product_49948298.jpg', Label: 3
Nom de l'image: b'image_881487935_product_126111824.jpg', Label: 3


In [38]:
for img_name in image_names[:5]:  # Vérifier les 5 premiers noms d'images
    img_path = os.path.join(image_dir, img_name)
    if not os.path.exists(img_path):
        print(f"Image non trouvée : {img_path}")
    else:
        print(f"Image trouvée : {img_path}")


Image trouvée : batchs_équilibrés\batch_entrainement\image_874729307_product_74244625.jpg
Image trouvée : batchs_équilibrés\batch_entrainement\image_874730792_product_49132716.jpg
Image trouvée : batchs_équilibrés\batch_entrainement\image_874731199_product_74093483.jpg
Image trouvée : batchs_équilibrés\batch_entrainement\image_874731370_product_49948298.jpg
Image trouvée : batchs_équilibrés\batch_entrainement\image_881487935_product_126111824.jpg


In [39]:
def load_image(img_name, label):
    try:
        # Nettoyage plus strict des caractères
        img_name = img_name.numpy().decode('utf-8', 'ignore')  # Décoder en utf-8 et ignorer les erreurs
        
        img_path = os.path.join(image_dir, img_name)  # Assurer le bon chemin d'accès
        img = tf.io.read_file(img_path)  # Lire l'image avec TensorFlow
        img = tf.image.decode_jpeg(img, channels=3)  # Décoder l'image (JPEG) en un Tensor
        img = tf.image.resize(img, [224, 224])  # Redimensionner l'image à 224x224
        img = img / 255.0  # Normalisation des pixels entre 0 et 1
        return img, label
    except tf.errors.NotFoundError:
        print(f"Image non trouvée : {img_name}")  # Afficher une erreur si l'image n'est pas trouvée
        return None, label
    except UnicodeDecodeError as e:
        print(f"Erreur de décodage avec le fichier {img_name}: {e}")
        return None, label
    except Exception as e:
        print(f"Erreur lors du chargement de l'image {img_name}: {e}")
        return None, label
